# Import
Ta import các thư viện cần thiết

In [ ]:
import requests
import base64
import time
import pandas as pd
import csv
import json
import re
import random
from requests import post, get

# API KEY
Với mỗi `App` của mỗi account, có số lần request giới hạn trong một khoảng thời gian. Nếu vượt qua số lần request thì sẽ bị lỗi rate-limit. Vì vậy, ta chuẩn bị nhiều API KEY để có thể tiếp tục lấy dữ liệu khi gặp lỗi. Đồng thời dùng cấu trúc try, except cũng như lưu dataframe sau một khoảng thời gian để hạn chế bị lỗi dẫn đến mất dữ liệu lấy được.

In [ ]:
CURRENT_CLIENT_ID = 1
CLIENT_CREDENTIALS = {
    1 : {
        'client_id': '489161f1d161402ca00a10e470b9b808',
        'client_secret': '6041044fc5dc498480994298ed0163c8'
    },
    2: {
        'client_id': 'dc4c9136605a478e9c836c59691cd474', 
        'client_secret': '7dca333866ab4a05be49bd4078d98744'
    },
    3: {
        'client_id': 'b6b19e48101040f5943ce1050a04b153', 
        'client_secret': '9182c41a5f4746d980130a3270823557'
    },
    4: {
        'client_id': 'cf1516d021e2489daf208a4f973544d2', 
        'client_secret': 'ebfc8b2428b94aa8977b0b34ad9a518f'
    },
    5: {
        'client_id': '106f35d5e02347749a5d11967e00a5f2', 
        'client_secret': 'fc971fb3df9b4710b4b73cffb4172a85'
    },
    6: {
        'client_id': 'eeda886daea447de937b498eeab2ca24', 
        'client_secret': '2f62c397322d4dcc8efdf53a2e0db85c'
    },
    7: {
        'client_id': '464d8814654d437683582a5c189a6e82', 
        'client_secret': 'aed59a52534a404597f37d735acbecfb'
    },
    8: {
        'client_id': '9f89459be490410a89ea56ee73caa09d', 
        'client_secret': 'a96d44704bcc4f879c446125b0963395'
    },
    9: {
        'client_id': 'f456115fa6df45a3bc04469a5a4d4384', 
        'client_secret': 'caec468cc4ca40bcbf6012c2ba4afea4'
    },
    10: {
        'client_id': '97961af1175d4bc28ba5f9d1aae89598', 
        'client_secret': 'a045e19fe1d24e549a7fdc9344f37ffe'
    }
}

# Token Handle
Các hàm xử lý với token API spotify

In [ ]:
def get_token(client_id, client_secret):
    # Mã hóa client_id và client_secret thành base64
    auth_string = f"{client_id}:{client_secret}"
    auth_bytes = auth_string.encode('utf-8')
    auth_base64 = base64.b64encode(auth_bytes).decode('utf-8')

    # Thiết lập thông tin yêu cầu
    auth_url = 'https://accounts.spotify.com/api/token'
    headers = {
        'Authorization': f'Basic {auth_base64}'
    }
    data = {
        'grant_type': 'client_credentials'
    }

    # Gửi yêu cầu POST để lấy token
    response = post(auth_url, headers=headers, data=data)

    # Kiểm tra phản hồi và trả về token nếu thành công
    if response.status_code == 200:
        token = response.json().get('access_token')
        return token
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def get_valid_token(client_id, client_secret):
    token = None
    while token is None:
        token = get_token(client_id, client_secret)
        if token is None:
            print("Token expired or error. Retrying in 1 minute...")
            time.sleep(60)  # Nếu token không hợp lệ, chờ 1 phút và thử lại
    return token

In [ ]:
def new_token(id):
    try:
        credentials = CLIENT_CREDENTIALS[id]
        client_id = credentials['client_id']
        client_secret = credentials['client_secret']
        
        return get_valid_token(client_id, client_secret)
        
    except KeyError:
        print(f"Error id: {id}")
        raise KeyError(f"No credentials found for app: {id}")
    except requests.exceptions.RequestException as e:
        raise Exception(f"Failed to get token: {str(e)}")

In [ ]:
token = new_token(CURRENT_CLIENT_ID)
print(token)

# Get track ids
Spotify không cung cấp api để lấy trực tiếp các track id. Vì thế ta thực hiện gián tiếp qua các bước:
- Get Categories: Ta tìm các danh mục theo một số ngôn ngữ bằng [API của spotify](https://developer.spotify.com/documentation/web-api/reference/get-categories)
- Get Playlist: Từ những categories vừa lấy được phía trên, ta xem mỗi categories như một keyword. Tìm kiếm các playlist liên quan dựa theo keyword. [Xem tại đây](https://developer.spotify.com/documentation/web-api/reference/search)
- Get Track ID: Lấy thông tin các bài nhạc trong playlist. [Xem tại đây](https://developer.spotify.com/documentation/web-api/reference/get-playlists-tracks)

*Lưu ý:*
- Trong [search API](https://developer.spotify.com/documentation/web-api/reference/search) có thể search theo tracks, tuy nhiên thực nghiệm với các kết quả từ categories thì không tốt bằng lấy playlist. Vì vậy ta dùng một bước gián tiếp như trên để cho kết quả tốt hơn.

## Get Categories
[API của spotify](https://developer.spotify.com/documentation/web-api/reference/get-categories) trả về các danh mục dựa theo ngôn ngữ nhất định. Thực tế thì categories đều có nghĩa giống nhau, tuy nhiên được dịch ra các ngôn ngữ tương ứng. Vì vậy ta có thể lấy được các playlist phổ biến ở nhiều vùng quốc gia khác nhau.

In [ ]:
locales = [
    "en_US", "en_GB", "sv_SE", "fr_FR", "de_DE", 
    "es_ES", "es_MX", "it_IT", "zh_CN", "zh_TW",
    "ja_JP", "ko_KR", "pt_PT", "pt_BR", "ru_RU",
    "ar_SA", "nl_NL", "no_NO", "da_DK"
]

In [ ]:
def fetch_all_category_names(auth_token, locale=None, total_names=1000):
    global CURRENT_CLIENT_ID
    url = "https://api.spotify.com/v1/browse/categories"
    headers = {
        "Authorization": f"Bearer {auth_token}"
    }

    limit = 50  # Maximum items per request
    offset = 0
    category_names = []
    global start_time

    while len(category_names) < total_names:
        params = {
            "locale": locale,
            "limit": limit,
            "offset": offset
        }
        
        if time.time() - start_time >= 2700:
            client_id, client_secret = CLIENT_CREDENTIALS[CURRENT_CLIENT_ID].values()
            token = get_valid_token(client_id, client_secret)
            headers["Authorization"] = f"Bearer {token}"
            start_time = time.time()
            
        response = get(url, headers=headers, params=params)
        if response.status_code == 429: # Ta xử lý cẩn thận với trường hợp rate-limit
            trying = 5
            while response.status_code == 429 and trying > 0:
                print("Rate limit exceeded. Change API KEY")
                try:
                    CURRENT_CLIENT_ID += 1
                    token = new_token(CURRENT_CLIENT_ID)
                except KeyError:
                    print(f"Error id: {CURRENT_CLIENT_ID}")
                    raise KeyError(f"No credentials found for app: {CURRENT_CLIENT_ID}")

                headers["Authorization"] = f"Bearer {token}"
                response = get(url, headers=headers, params=params)
                trying -= 1
            if trying == 0:
                print("Rate limit exceeded. Try again later.") 
                response.raise_for_status()
        
        data = response.json()
        
        try:
            categories = data['categories']['items']
            if not categories:
                break
            
            # Extract names and add them to the result list
            category_names.extend([category['name'] for category in categories])
            
            # Update offset for the next page
            offset += limit
            
            # Stop if we reach the total_names limit
            if len(category_names) >= total_names:
                category_names = category_names[:total_names]  # Truncate excess names
                break
        except KeyError:
            raise ValueError("Invalid response format: 'categories' or 'items' key is missing.")
    
    return category_names

In [ ]:
token = new_token(CURRENT_CLIENT_ID)
start_time = time.time()

keywords = []
for locale in locales:
    try:
        print(f"Fetching categories for locale: {locale}")
        categories = fetch_all_category_names(auth_token=token, locale=locale)
        keywords.extend(categories)
        print(f"{len(categories)} categories fetched for locale")
    except Exception as e:
        print(f"Error fetching categories for locale {locale}: {e}")

In [ ]:
print(len(keywords))

## Get Playlist

In [ ]:
def search_playlists_by_keyword(token, keyword, limit=50):
    global CURRENT_CLIENT_ID
    url = f"https://api.spotify.com/v1/search?q={keyword}&type=playlist&limit={limit}"
    headers = {
        "Authorization": f"Bearer {token}"
    }
    result = get(url, headers=headers)
    
    if result.status_code == 429:
        trying = 5
        while result.status_code == 429 and trying > 0:
            print("Rate limit exceeded. Change API KEY")
            try:
                CURRENT_CLIENT_ID += 1
                token = new_token(CURRENT_CLIENT_ID)
            except KeyError:
                print(f"Error id: {CURRENT_CLIENT_ID}")
                raise KeyError(f"No credentials found for app: {CURRENT_CLIENT_ID}")

            headers["Authorization"] = f"Bearer {token}"
            result = get(url, headers=headers)
            trying -= 1
        if trying == 0:
            print("Rate limit exceeded. Try again later.") 
            result.raise_for_status()
    elif result.status_code != 200:
        print(f"Failed to fetch playlists for keyword '{keyword}'. Status Code: {result.status_code}")
        return []

    data = json.loads(result.content)
    return data["playlists"]["items"]

In [ ]:
def remove_duplicate_playlists(playlists):
    """Loại bỏ playlist trùng lặp dựa trên playlist ID."""
    unique_playlists = {}
    for playlist in playlists:
        if not playlist:  # Bỏ qua nếu playlist là None
            continue
        playlist_id = playlist.get("id")
        if playlist_id not in unique_playlists:
            unique_playlists[playlist_id] = playlist
    return list(unique_playlists.values())

In [ ]:
token = new_token(CURRENT_CLIENT_ID)
start_time = time.time()

# Lấy playlist từ các từ khóa
playlists = []
for keyword in keywords:
    if time.time() - start_time >= 2700:
        token = new_token(CURRENT_CLIENT_ID)
        start_time = time.time()
    playlists.extend(search_playlists_by_keyword(token, keyword, limit=5))
    time.sleep(0.2)  # Nghỉ để tránh vượt quá hạn mức API

# Loại bỏ các playlist trùng lặp
unique_playlists = remove_duplicate_playlists(playlists)

## Get Track ID
Với mục đích là lấy càng nhiều càng tốt, ta nên sử dụng cấu trúc try-except cũng như lưu data thu thập được sau một khoảng thời gian nhất định để tránh mất dữ liệu thu thập được bởi các lỗi bất ngờ.

In [ ]:
def get_all_tracks_from_playlist(token, playlist_id):
    global CURRENT_CLIENT_ID
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    headers = {
        "Authorization": f"Bearer {token}"
    }
    all_tracks = []
    
    while url:
        result = get(url, headers=headers)
        
        if result.status_code == 429:
            trying = 5
            while result.status_code == 429 and trying > 0:
                print("Rate limit exceeded. Change API KEY")
                try:
                    CURRENT_CLIENT_ID += 1
                    token = new_token(CURRENT_CLIENT_ID)
                except KeyError:
                    print(f"Error id: {CURRENT_CLIENT_ID}")
                    raise KeyError(f"No credentials found for app: {CURRENT_CLIENT_ID}")
                
                headers["Authorization"] = f"Bearer {token}"
                result = get(url, headers=headers)
                trying -= 1
            if trying == 0:
                print("Rate limit exceeded. Try again later.") 
                result.raise_for_status()

        elif result.status_code != 200:
            print(f"Failed to fetch tracks for playlist {playlist_id}. Status Code: {result.status_code}")
            return all_tracks  # Trả về các tracks đã lấy được trước đó (nếu có)
        
        data = json.loads(result.content)
        all_tracks.extend(data["items"])  # Thêm tất cả các bài hát vào danh sách
        
        # Lấy URL của trang tiếp theo nếu có
        url = data.get("next")
    
    return all_tracks

In [ ]:
csv_file = "track_ids.csv"

# Ghi dữ liệu vào file CSV
with open(csv_file, mode="w", newline="", encoding="utf-8-sig") as file:
    writer = csv.writer(file)
        
    # Ghi tiêu đề cột
    writer.writerow([
        "Playlist Name", "Playlist Owner", "Track Index", "Track Name",
        "Track ID", "Artist Name", "Album Name", "Track URI", "Artist URI",
        "Album URI", "Duration (ms)"
    ])
        
    # Lấy thông tin bài hát từ mỗi playlist
    for idx, playlist in enumerate(unique_playlists):
        playlist_name = playlist.get("name", "Unknown Playlist")
        playlist_owner = playlist.get("owner", {}).get("display_name", "Unknown Owner")
        playlist_id = playlist.get("id", None)

        if not playlist_id:
            print(f"{idx + 1}. Skipped playlist with missing ID")
            continue

        # Lấy tất cả các bài hát từ playlist này
        if time.time() - start_time >= 2700:
            token = new_token(CURRENT_CLIENT_ID)
            start_time = time.time()
                
        tracks = get_all_tracks_from_playlist(token, playlist_id)

        # Lưu thông tin từng bài hát vào file CSV
        for track_idx, item in enumerate(tracks):
            track = item.get("track")
            if not track:
                print(f"      {track_idx + 1}. Invalid track (NoneType)")
                continue

            # Lấy thông tin track và kiểm tra các giá trị
            track_name = track.get("name", "Unknown Track")
            track_id = track.get("id", "Unknown ID")
            artist_name = track.get("artists", [{}])[0].get("name", "Unknown Artist")
            track_uri = track.get("uri", "Unknown URI")
            artist_uri = track.get("artists", [{}])[0].get("uri", "Unknown URI")
            album_uri = track.get("album", {}).get("uri", "Unknown URI")
            duration_ms = track.get("duration_ms", "Unknown Duration")
            album_name = track.get("album", {}).get("name", "Unknown Album")

            # Ghi dữ liệu bài hát vào file CSV
            writer.writerow([
                playlist_name, playlist_owner, track_idx + 1, track_name,
                track_id, artist_name, album_name, track_uri, artist_uri,
                album_uri, duration_ms
            ])

print(f"\nData saved to {csv_file}")

# Get features
Ta dùng [API](https://developer.spotify.com/documentation/web-api/reference/get-several-audio-features) để lấy các features từng bài hát.

*Lưu ý:*
- Hiện tại endpoint này đã đang bị đóng trong quá trình biên soạn lại nội dung cho file notebook này.
- Code mang tính tham khảo nếu spoify mở lại API này.

In [ ]:
def save_to_csv(data, filename='audio_features.csv'):
    if data is not None:  # Kiểm tra xem dữ liệu có phải là None không
        try:
            # Kiểm tra dữ liệu có trong dạng đúng không
            if isinstance(data, list) and isinstance(data[0], dict):
                df = pd.DataFrame(data)
                df.to_csv(filename, mode='a', header=not pd.io.common.file_exists(filename), index=False)
                print(f"Data saved to {filename}")
            else:
                print("Data format is incorrect.")
        except Exception as e:
            print(f"Error saving to CSV: {e}")
    else:
        print("No data to save.")

In [ ]:
def get_audio_features(track_ids, token):
    url = 'https://api.spotify.com/v1/audio-features'
    
    # Chuyển tất cả các Track ID thành chuỗi
    track_ids = map(str, track_ids)
    
    # Ghép các Track ID thành chuỗi
    ids = ','.join(track_ids)
    
    headers = {'Authorization': f'Bearer {token}'}
    
    response = requests.get(f'{url}?ids={ids}', headers=headers)

    if response.status_code == 200:
        return response.json()  # Trả về dữ liệu JSON nếu thành công
    elif response.status_code == 429:
        print("Rate limit exceeded. Change API KEY.")
        trying = 5
        while response.status_code == 429 and trying > 0:
            
            try:
                CURRENT_CLIENT_ID += 1
                token = new_token(CURRENT_CLIENT_ID)
            except KeyError:
                print(f"Error id: {CURRENT_CLIENT_ID}")
                raise KeyError(f"No credentials found for app: {CURRENT_CLIENT_ID}")
            
            headers['Authorization'] = f'Bearer {token}'
            response = requests.get(f'{url}?ids={ids}', headers=headers)
            trying -= 1
        if trying == 0:
            print("Rate limit exceeded. Try again later.")
            return None
    else:
        print(f"Error fetching data: {response.status_code}")
        return None  # Trả về None nếu có lỗi

# Hàm xử lý các track từ CSV
def process_tracks_from_csv(file_path):
    # Lấy token từ client_id và client_secret
    global CURRENT_CLIENT_ID
    token = new_token(CURRENT_CLIENT_ID)
    start_time = time.time()
    
    if token is None:
        print("Failed to get token.")
        return
    
    track_ids = []  # Danh sách track ids từ CSV
    chunk_size = 100  # Số lượng track_id mỗi lần gọi API
    
    # Đọc file CSV và lấy Track ID
    df = pd.read_csv(file_path)
    track_ids = df['Track ID'].tolist()
    # Xử lý theo từng chunk
    for i in range(0, len(track_ids), chunk_size):
        if time.time() - start_time >= 2700: # 45 minutes
            token = new_token(CURRENT_CLIENT_ID)
            start_time = time.time()
            
        track_ids_chunk = track_ids[i:i+chunk_size]
        
        audio_features = get_audio_features(track_ids_chunk, token)
        
        if audio_features:
            save_to_csv(audio_features['audio_features'])  # Lưu dữ liệu vào CSV
            #print(audio_features['audio_features'])
        else:
            print("No data returned for the chunk.")

In [ ]:
file_path = 'track_ids.csv' # Đường dẫn đến file track_ids.csv vừa thu thập phía trên
process_tracks_from_csv(file_path)

# Merged CSV
Ta gộp file `track_ids.csv` và file `audio_features.csv`, ta được file `merged_tracks.csv`.

In [ ]:
def merge_files(input_csv, feature_csv, merged_csv):
    # Đọc file đầu vào
    input_data = pd.read_csv(input_csv)
    feature_data = pd.read_csv(feature_csv)

    # Đổi tên cột `id` trong feature_data để khớp với `Track ID` trong input_data
    feature_data.rename(columns={'id': 'Track ID'}, inplace=True)

    # Gộp hai file dựa trên `Track ID`
    merged_data = pd.merge(input_data, feature_data, on='Track ID', how='inner')

    # Loại bỏ các bản ghi trùng lặp dựa trên 'Track ID'
    merged_data = merged_data.drop_duplicates(subset=['Track ID'])

    # Ghi file kết quả
    merged_data.to_csv(merged_csv, index=False)
    print(f"Merged data has been saved to {merged_csv}")

In [ ]:
merge_files("track_ids.csv", "audio_features.csv", "merged_tracks.csv")

# Get other features
từ file csv có được ở các bước trên. Ta cần thêm một số thuộc tính sau:
- popularity
- release day

Dùng [API](https://developer.spotify.com/documentation/web-api/reference/get-track) sau để tiến thành thu thập.

In [ ]:
def fetch_data(id_chunk, token):
    global CURRENT_CLIENT_ID
    url = f"https://api.spotify.com/v1/tracks?ids={id_chunk}"
    headers = {
        "Authorization": f"Bearer {token}"
    }
    
    while True:
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                data = response.json()
                return {
                    track['id']: {
                        'popularity': track['popularity'],
                        'release_date': track['album']['release_date']
                    }
                    for track in data['tracks'] if 'popularity' in track and 'album' in track
                }
            elif response.status_code == 429:  # Rate limit
                CURRENT_CLIENT_ID += 1
                new_token_value = new_token(CURRENT_CLIENT_ID)
                if new_token_value:
                    token = new_token_value
                    continue
                else:
                    raise Exception("Failed to get valid token")
            else:
                response.raise_for_status()
                
        except requests.exceptions.RequestException as e:
            if hasattr(e.response, 'status_code') and e.response.status_code == 429:
                continue
            raise

# Hàm xử lý chunk và thêm popularity + release_date
def process_chunk(chunk, token):
    ids = chunk['Track ID'].tolist()
    id_chunks = [",".join(ids[i:i+50]) for i in range(0, len(ids), 50)]
    data = {}
    for id_chunk in id_chunks:
        data.update(fetch_data(id_chunk, token))
    
    # Thêm cột popularity và release_date vào chunk
    chunk['popularity'] = chunk['Track ID'].map(lambda x: data.get(x, {}).get('popularity', None))
    chunk['release_date'] = chunk['Track ID'].map(lambda x: data.get(x, {}).get('release_date', None))
    return chunk

In [ ]:
def save_progress(filtered_songs, output_csv):
    """Save current progress to CSV"""
    if filtered_songs:
        pd.concat(filtered_songs).to_csv(output_csv, index=False)
        print(f"Progress saved to {output_csv}")

In [ ]:
def get_track(input_csv, output_csv, target_count = 10e9):
    global CURRENT_CLIENT_ID
    filtered_songs = []
    token = new_token(CURRENT_CLIENT_ID)
    
    try:
        successfull = 1
        for chunk in pd.read_csv(input_csv, chunksize=50):
            try:
                # If token is invalid, get new one
                if not token:
                    token = new_token(CURRENT_CLIENT_ID)
                    if not token:
                        raise Exception("Failed to get valid token")
                
                chunk = process_chunk(chunk, token)
                filtered_songs.append(chunk)
                print(f"{successfull} chunk append sucessfully, id = {CURRENT_CLIENT_ID}")
                successfull += 1
                
                if sum(len(f) for f in filtered_songs) >= target_count:
                    break
                    
            except requests.exceptions.HTTPError as e:
                if e.response.status_code == 429:  # Rate limit
                    save_progress(filtered_songs, output_csv)
                    CURRENT_CLIENT_ID += 1
                    token = new_token(CURRENT_CLIENT_ID)
                    continue
                raise
                
    except Exception as e:
        print(f"Error occurred: {e}")
        save_progress(filtered_songs, output_csv)
        return
        
    # Final save
    filtered_songs.to_csv(output_csv, index=False)
    print(f"Completed! Saved {len(filtered_songs)} songs to {output_csv}")

In [ ]:
get_track("merged_tracks.csv", "music_data.csv")